# HIV-1 Protease-substrates Near Attack Conformation Analysis

Author: S. Kashif Sadiq

Correspondence: kashif.sadiq@embl.de, Affiliation: 1. Heidelberg Institute for Theoretical Studies, HITS gGmbH 2. European Moelcular Biology Laboratory

This notebook contains molecular dynamics (MD) simulation analyses of HIV-1 protease subtrate near attack conformations for the manuscripts:


S.K. Sadiq‡ (2020) Catalysts, Fine-tuning of sequence-specificity by near attack conformations in enzyme-catalyzed peptide hydrolysis, 10 (6) 684

Sadiq, S.K. ‡ and Coveney P.V. (2015). J Chem Theor Comput. Computing the role of near attack conformations in an enzyme-catalyzed nucleophilic bimolecular reaction. 11 (1), pp 316–324



### Initial Module Import

In [ ]:
#####################################################

from nac import *

import sys
import glob
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit
from scipy.integrate import odeint
import matplotlib.pyplot as plt
from matplotlib import gridspec
#from lmfit import minimize, Parameters, Parameter, report_fit
#import numdifftools
from scipy.special import lambertw
import math
import os
from distutils.version import LooseVersion
from scipy.stats import norm
from sklearn.neighbors import KernelDensity
import itertools
from scipy.signal import argrelextrema

#####################################################

### Load MD postprocessed data 

#### Protein-Substrate complex systems

In [ ]:
# columns in data files are as followS:
# NWI $NWD $ATTACK $ATTACKA 
# $H1d $H1a (D25:O2 - D25:H - p1:O)
#$H21d $H21a $H22d $H22a (D25:O2 - D25:H - D25':O1, D25:O2 - D25:H - D25':O2)
#$H31d $H31a $H32d $H32a (p1':N - p1':H - D25':O1, p1':N - p1':H - D25':O2)
# $H411d $H411a $H412d $H412a (WAT:O - WAT:H1 - D25':O1, WAT:O - WAT:H2 - D25':O1)
#$H421d $H421a $H422d $H422a (WAT:O - WAT:H1 - D25':O2, WAT:O - WAT:H2 - D25':O2)
# $H511d $H511a $H512d $H512a (WAT:O - WAT:H1 - D25:O1, WAT:O - WAT:H2 - D25:O1)
#$H521d $H521a $H522d $H522a (WAT:O - WAT:H1 - D25:O2, WAT:O - WAT:H2 - D25:O2)
#In this terminology the monoprotonated aspartyl is on D25:O2

#List of substrates systems
syslist=['maca','cap2','p2nc','ncp1','p1p6','p6pr','prrt','rtrh','rhin']

#Directory where each of the enzyme=bound substrate sub-directories are located
data_dir='./data/enz/'

#Data set is in format 1-x.dat, where x is the sim number. Function loads the first 100 sims into the dataset
alldata=load_dataset_from_simlist(data_dir, syslist,100)
print(np.shape(alldata))

#### Apo Substrate systems

In [ ]:
# columns in data files are as followS:
# NWI $NWD $ATTACK $ATTACKA

#Directory where each of the apo substrate sub-directories are located
data_dir='./data/apo/'

#Data set is in format x-w.dat, where x is the sim number, w is the sequential part of the simulation (100 parts per sim). Function loads the first 10 sims into the dataset
apodata=load_dataset_from_range(data_dir, syslist, 10, 100)
print(np.shape(apodata))

### Parameter  and Constants definition

In [ ]:
#Constants
#kB = 0.001987191;
#kBT= kB*298;
#kBT_over_h = 6.24*(10**12) #s^-1

#Define Near water, Burgi Dunitz thresholds and bootstrapping parameters

#Near water binding threshold
#nwb_cut=[4.6, 3.9, 4.6, 4.6, 4.0, 4.6, 4.2, 4.9, 4.6]
nwb_cut=[4.6, 4.2, 4.6, 4.6, 4.6, 4.6, 4.6, 4.6, 4.6]
nwb_cut_apo=6.0*np.ones((1,9))[0]

#Burgi-Dunitz criteria
BDdist=3.2;
BDanglow=100;
BDanghigh=110;

kBT, kBT_over_h=kBT_func(300)
print(kBT, kBT_over_h)

## Nearest Water molecule distance distributions

#### Individual systems

In [ ]:
#For apo substrate data take every 10th step
#Xdata=apodata[9:-1:10,:,:]
#For enzyme-substrate data take all steps
Xdata=alldata
#Define data column
colno=1
#Define system
junction_list=['MA-CA','CA-SP1','SP1-NC','NC-SP2','SP2-p6','TFR-PR','PR-RT','RT-RH','RH-IN']
sys=0

#Partition line
#Apo
#xpoint=nwb_cut_apo[sys]*np.ones((1,12))[0]
#Enzyme-sub
xpoint=nwb_cut[sys]*np.ones((1,12))[0]

#Plot distribution
fig, axes = plt.subplots(1, 1, sharex=True,sharey=True,figsize=(8,6))
plt,log_dens = plot_distribution_single_panel(plt, axes, Xdata,junction_list,sys,colno,xpoint,xlab=r'$d_{n} (\AA)$',ylab=r'$d_{n} (\AA)$')


#### Catalyzed Systems - Enzyme-substrate complexes

In [ ]:
junction_list=['MA-CA','CA-SP1','SP1-NC','NC-SP2','SP2-p6','TFR-PR','PR-RT','RT-RH','RH-IN']
Xdata=alldata
xpoint=nwb_cut[sys]*np.ones((1,12))[0]
X_plot = np.linspace(0, 10, 1000)[:, np.newaxis]
colno=1

#Plot figure
fig, axes = plt.subplots(3, 3, sharex=True,sharey=True,figsize=(12,9))
fig.subplots_adjust(hspace=0.00, wspace=0.00)
for i in range(3):
    for j in range(3):
        sys=3*i + j        
        # Calculate distribution and plot
        plt,log_dens = plot_distribution_multi_panel(plt, axes, Xdata,junction_list,sys,colno,xpoint,i,j)
        #Calculate frequency curve
        if sys==0:
            freq = np.exp(log_dens)[:,np.newaxis]
        else:
            freq=np.hstack((freq,np.exp(log_dens)[:,np.newaxis]))

#Plot master labels for multi panel figure
plt.rc('text', usetex=True)  
axes[2,1].set_xlabel(r'$d_{n}$ (\AA)',fontsize=30)
axes[1,0].set_ylabel(r'$\rho(d_{n})$',fontsize=30)

#Save figure
#fig.savefig('figures/dn_distrib.png')
#fig.savefig('figures/dn_distrib.eps')

### Maxima of density distribution
#Peak analysis - determines all the peaks in the distribution plots
maxima=[]
for i in range(np.shape(freq)[1]):
    # for local maxima
    maxima.append(np.transpose(X_plot[argrelextrema(freq[:,i], np.greater)[0]]))

#Print x value where maxima are located
for x in maxima:
    print(x)


#### Non-Catalyzed Systems - Apo substrates

In [ ]:
junction_list=['MA-CA','CA-SP1','SP1-NC','NC-SP2','SP2-p6','TFR-PR','PR-RT','RT-RH','RH-IN']
Xdata=apodata[9:-1:10,:,:]
xpoint=nwb_cut_apo[sys]*np.ones((1,12))[0]
X_plot = np.linspace(0, 10, 1000)[:, np.newaxis]
colno=1


#Plot figure 
fig, axes = plt.subplots(3, 3, sharex=True,sharey=True,figsize=(12,9))
fig.subplots_adjust(hspace=0.00, wspace=0.00)
for i in range(3):
    for j in range(3):
        sys=3*i + j
        
        # Calculate distribution and plot
        plt,log_dens = plot_distribution_multi_panel(plt, axes, Xdata,junction_list,sys,colno,xpoint,i,j)
        #Calculate frequency curve
        if sys==0:
            freq2 = np.exp(log_dens)[:,np.newaxis]
        else:
            freq2=np.hstack((freq2,np.exp(log_dens)[:,np.newaxis]))

#Plot master labels for multi panel figure
plt.rc('text', usetex=True)  
axes[2,1].set_xlabel(r'$d_{o}$ (\AA)',fontsize=30)
axes[1,0].set_ylabel(r'$\rho(d_{o})$',fontsize=30)

#Save figure
#fig.savefig('figures/do_distrib.eps')

### Maxima of density distribution
#Peak analysis
maxima=[]
for i in range(np.shape(freq2)[1]):
    # for local maxima
    maxima.append(np.transpose(X_plot[argrelextrema(freq2[:,i], np.greater)[0]]))

#Print x value where maxima are located
for x in maxima:
    print(x)


# for local minima
#argrelextrema(x, np.less)

### Computation of NAC statistics and Thermodynamic quantities from complete data set

This analysis computes quantities using the entire data set - without using subset samples to generate mean and standard deviations

#### Individual Systems

In [ ]:
#Total data set free energies for individual system

sys=0 #MACA system

#Enzyme-substrate system
nwb_cutoff=nwb_cut[sys]
data=alldata[:,:,sys]
#Apo system
#nwb_cutoff=6.0
#data=apodata[:,:,sys]

N,count_nwb, count_nac, rho_nwb, rho_nac, G_nwb, G_nac = NAC_calc(data,nwb_cutoff,BDdist,BDanglow,BDanghigh,kBT)

print(N,count_nwb, count_nac, rho_nwb, rho_nac, G_nwb, G_nac)


#### Catalyzed Systems - Enzyme-substrate complexes

In [ ]:
#Total data set free energies for all systems
np.set_printoptions(precision=4)
totGnac=total_Gnac(alldata,nwb_cut,BDdist,BDanglow,BDanghigh,kBT)
print(totGnac)


#### Non-Catalyzed Systems - Apo substrates

In [ ]:
#Total data set free energies for all systems
np.set_printoptions(precision=4)
nwb_cut_apo=6.0*np.ones((1,9))[0]
totGnac=total_Gnac(apodata,nwb_cut_apo,BDdist,BDanglow,BDanghigh,kBT)
print(totGnac)

In [ ]:
print(np.mean(totGnac[:,-1]),np.std(totGnac[:,-1]))

### Computation of NAC free energy as a function of partition distance between bound and unbound nearest water molecules

#### Catalyzed Systems - Enzyme-substrate complexes

In [ ]:
# NAC free energy as function of near water distance threshold
# USing Total data set - so without error bars
min_dn=30
max_dn=71
Gvary=calc_Gvary(alldata,min_dn,max_dn,BDdist,BDanglow,BDanghigh,kBT)   

#Plot G_NAC as a function of partition distance
#xlim=[2.8,5]
#ylim=[1.5,8]
#plot_Gvary(plt,Gvary,min_dn,max_dn,xlim,ylim,apo=False)
#plt.savefig('figures/g_versus_dnp.eps')

#### Non-Catalyzed Systems - Apo substrates

In [ ]:
# NAC free energy as function of near water distance threshold
# USing Total data set - so without error bars
min_dn=30
max_dn=71
Gvary_apo=calc_Gvary(apodata,min_dn,max_dn,BDdist,BDanglow,BDanghigh,kBT)   

#Plot G_NAC as a function of partition distance
#xlim=[2.8,7]
#ylim=[1.5,5]
#plot_Gvary(plt,Gvary_apo,min_dn,max_dn,xlim,ylim,apo=True)
#plt.savefig('figures/g_versus_dnp_apo.eps')


### Plot both complex and apo systems

In [ ]:
junction_list=['MA-CA','CA-SP1','SP1-NC','NC-SP2','SP2-p6','TFR-PR','PR-RT','RT-RH','RH-IN']
min_dn=30
max_dn=71
dn=[x/10 for x in range(min_dn,max_dn)]
color_list=['k','r','b','g','m','y','c','k','r']
marker_list=['o','o','o','o','o','s','s','s','s']

fig = plt.figure(figsize=(18, 6)) 
fig.subplots_adjust(hspace=0.05, wspace=0.3)
gs = gridspec.GridSpec(1, 2, width_ratios=[1, 1])
gs.update(left=0.1,right=0.9,top=0.95,bottom=0.2,wspace=0.3,hspace=0.05)
ax0 = plt.subplot(gs[0])
ax1 = plt.subplot(gs[1])


#Figure A
xlim=[2.8,5]
ylim=[1.5,8]
G=Gvary[:,0:-1:2]
for i in range(np.shape(G)[0]):
    m = G[i,:]
    if i<5:
        ax0.plot(dn[0:-1:2], m, ls='-', label=junction_list[i], color=color_list[i],marker=marker_list[i], ms=10, mec=color_list[i], mfc=color_list[i])
    else:
        ax0.plot(dn[0:-1:2], m, ls='-', color=color_list[i],marker=marker_list[i], ms=10, mec=color_list[i], mfc=color_list[i])


plt.rc('text', usetex=True)        
ax0.legend(loc='best',fontsize = 20)
ax0.set_xlabel(r'$d^{p}_{n}$ (\AA)',fontsize=30)
ax0.set_ylabel(r'$\Delta G^{\ddagger c}_{N}$ (kcal/mol)',fontsize=30)
ax0.set_xticks([x/10 for x in range(30,70,5)])    
ax0.set_yticks([y/10 for y in range(15,70,10)])
ax0.tick_params(axis='x', labelsize=30)
ax0.tick_params(axis='y', labelsize=30)
ax0.set_xlim(xlim[0],xlim[1])
ax0.set_ylim(ylim[0],ylim[1])
ax0.text(3.0, 7.0, '(A)', fontsize=30)

#Figure B
xlim=[2.8,7]
ylim=[1.5,5]

G=Gvary_apo[:,0:-1:2]
for i in range(np.shape(G)[0]):
    m = G[i,:]
    if i>=5:
        ax1.plot(dn[0:-1:2], m, ls='-', label=junction_list[i], color=color_list[i],marker=marker_list[i], ms=10, mec=color_list[i], mfc=color_list[i])
    else:
        ax1.plot(dn[0:-1:2], m, ls='-', color=color_list[i],marker=marker_list[i], ms=10, mec=color_list[i], mfc=color_list[i])


ax1.legend(loc='best',fontsize = 20)
ax1.set_xlabel(r'$d^{p}_{o}$ (\AA)',fontsize=30)
ax1.set_ylabel(r'$\Delta G^{\ddagger u}_{N}$ (kcal/mol)',fontsize=30)
ax1.set_xticks([x/10 for x in range(30,70,5)])    
ax1.set_yticks([y/10 for y in range(15,70,10)])
ax1.tick_params(axis='x', labelsize=30)
ax1.tick_params(axis='y', labelsize=30)
ax1.set_xlim(xlim[0],xlim[1])
ax1.set_ylim(ylim[0],ylim[1])
ax1.text(3.0, 4.5, '(B)', fontsize=30)

#gs.update(left=0.1,right=0.9,top=0.965,bottom=0.03,wspace=0.3,hspace=0.09)


#fig.savefig('figures/g_versus_dnp_dno.eps')

#fig.savefig('figures/g_versus_dnp_dno_new.eps')

## Computation of NAC statistics and Thermodynamic quantities from data subset averaging

This analysis computes quantities using subset samples to generate mean and standard deviations

### Catalyzed Systems - Enzyme-substrate complexes

#### Individual system NWB and NAC free energy using averages of data subsets

In [ ]:
#Divide data into subsets
numsets=2
datasize=np.shape(alldata)[0]
range_list = subsets(numsets,datasize)
#range_list = bootstrapping(numsets,datasize)

sys=6
mat=[]
data = alldata[:,:,sys]
for i in range(numsets):
    N,count_nwb, count_nac, rho_nwb, rho_nac, G_nwb, G_nac = NAC_calc(data[range_list[i]],nwb_cut[sys],BDdist,BDanglow,BDanghigh,kBT)
    mat.append([count_nwb, count_nac, rho_nwb, rho_nac, G_nwb, G_nac])

mat=np.array(mat)

#This is calculated using numsets=2 for system6 because of low count number
G_m6=np.mean(mat, axis=0)
G_s6=np.std(mat, axis=0)

#print(G_m6)
#print(G_s6)

#### All systems NWB and NAC free energies using averages of data subsets

In [ ]:
numsets=5
datasize=np.shape(alldata)[0]
range_list = subsets(numsets,datasize)
#All systems NWB and NAC free energies using averages of data subsets
for j in range(np.shape(alldata)[2]):   
    mat=[]
    data = alldata[:,:,j]
    for i in range(numsets):
        N,count_nwb, count_nac, rho_nwb, rho_nac, G_nwb, G_nac = NAC_calc(data[range_list[i]],nwb_cut[j],BDdist,BDanglow,BDanghigh,kBT)
        mat.append([count_nwb, count_nac, rho_nwb, rho_nac, G_nwb, G_nac])    

    mat=np.array(mat)

    if j == 0:
        G_all = mat
    else:
        G_all=np.dstack((G_all,mat))

G_all_mean=np.transpose(np.mean(G_all,axis=0))
G_all_std=np.transpose(np.std(G_all,axis=0))        

#Replace results of system 6 with averaging from just 2 subsets
G_all_mean[6,-1]=G_m6[-1]
G_all_std[6,-1]=G_s6[-1]

print(G_all_mean)
print(G_all_std)



#### NAC Free energy as a function of partition distance - with shaded error region

In [ ]:
# NAC free energy as function of near water distance threshold - using data subsets so with mean and std
#min_dn=30
#max_dn=71
#Gvary_errs=calc_Gvary_witherrors(alldata,range_list,numsets,min_dn,max_dn,BDdist,BDanglow,BDanghigh,kBT)

#xlim=[2.8,5]
#ylim=[1.5,8]
#plot_Gvary_witherrors(plt,Gvary_errs,min_dn,max_dn,xlim,ylim)

### Non-Catalyzed Systems - Apo substrates

#### Individual system NWB and NAC free energy using averages of data subsets

In [ ]:
numsets=5
datasize=np.shape(apodata)[0]
range_list = subsets(numsets,datasize)
#range_list = bootstrapping(numsets,datasize)


sys=6
mat=[]
data = apodata[:,:,sys]

for i in range(numsets):
    N,count_nwb, count_nac, rho_nwb, rho_nac, G_nwb, G_nac = NAC_calc(data[range_list[i]],nwb_cut_apo[sys],BDdist,BDanglow,BDanghigh,kBT)
    mat.append([count_nwb, count_nac, rho_nwb, rho_nac, G_nwb, G_nac])

mat=np.array(mat)
print(mat)
print(np.mean(mat, axis=0))
print(np.std(mat, axis=0))

#### All systems NWB and NAC free energies using averages of data subsets

In [ ]:
for j in range(np.shape(apodata)[2]):   
    mat=[]
    data = apodata[:,:,j]
    for i in range(numsets):
        N,count_nwb, count_nac, rho_nwb, rho_nac, G_nwb, G_nac = NAC_calc(data[range_list[i]],nwb_cut_apo[j],BDdist,BDanglow,BDanghigh,kBT)
        mat.append([count_nwb, count_nac, rho_nwb, rho_nac, G_nwb, G_nac])    

    mat=np.array(mat)

    if j == 0:
        G_all = mat
    else:
        G_all=np.dstack((G_all,mat))

G_all_mean_apo=np.transpose(np.mean(G_all,axis=0))
G_all_std_apo=np.transpose(np.std(G_all,axis=0))        
print(G_all_mean_apo)
print(G_all_std_apo)

#### NAC Free energy as a function of partition distance - with shaded error region

In [ ]:
# NAC free energy as function of near water distance threshold - using data subsets so with mean and std
#min_dn=30
#max_dn=71
#Gvary_errs_apo=calc_Gvary_witherrors(apodata,range_list,numsets,min_dn,max_dn,BDdist,BDanglow,BDanghigh)

#xlim=[2.8,7]
#ylim=[1.5,7]
#plot_Gvary_witherrors(plt,Gvary_errs_apo,min_dn,max_dn,xlim,ylim, apo=True)

### Comparison of G_NAC for Catalyzed Enzyme-substrate systems with experimental data set

In [ ]:
#Comparison of experimental values with GNAC

junction_list=['MA-CA','CA-SP1','SP1-NC','NC-SP2','SP2-p6','TFR-PR','PR-RT','RT-RH','RH-IN']

#Using data in Maschera et al
#Then using ratio of NC-SP2/CA-SP1 in Feher and Toszer parameters to calculate NC-SP2 equivalent in Maschera conditions
kcat=[[23, 2], [2.6, 0.1], [41, 6], [5*2.6/3, 0.1], [0.25, 0.1], [43, 4], [20, 3], [1.8, 0.1], [5.1, 0.1]] #s^-1
kcat=np.array(kcat)         
Gcat=kBT*np.log(kBT_over_h/kcat[:,0])
comp=G_all_mean[:,-1]
comp_err=G_all_std[:,-1]


y_fit, popt, r2 = linear_fit(Gcat, comp)


print(Gcat)
print(comp)
print(comp_err)
print(Gcat-comp)
print(popt,r2)

print(30-Gcat)

### Plot of G_NAC vs experimental G_cat

In [ ]:
xlim=[14,19]
ylim=[0,6]
#ylim=[-4,2]

plt.figure(figsize=(12,6))
plt.rc('text', usetex=True)
#plt.plot(Gcat,comp,ls='',marker='o',ms=14, mec='k',mfc='r')
plt.errorbar(Gcat,comp,comp_err,ls='',color='k',marker='o',ms=14, mec='k',mfc='#b2182b')

plt.plot(Gcat, y_fit, '-k')

plt.text(18, 1, '$R^{2}$ =' +  "{:.2f}".format(r2), fontsize=30,color='k')

delx=[-0.2,-0.2,-0.2,-0.3,-0.2,-0.2,-0.1,0.1,-0.2]
dely=[0.2,0.2,0.2,0.3,0.2,-0.4,0.55,0,-0.4]
for i in range(len(Gcat)):
    plt.text(Gcat[i]+delx[i], comp[i]+dely[i], str(junction_list[i]),fontsize=15,color='k')

    
#plt.legend(loc='best',fontsize = 20)
plt.xlabel(r'$\Delta G^{\ddagger c}$ (kcal/mol)',fontsize=30)
plt.ylabel(r'$\Delta G^{\ddagger c}_{N}$ (kcal/mol)',fontsize=30)
plt.xticks([x/10 for x in range(140,190,10)], fontsize=30)    
plt.yticks([y/10 for y in range(0,60,10)], fontsize=30)
#plt.yticks([y/10 for y in range(-40,20,10)], fontsize=30)
plt.xlim(xlim[0],xlim[1])
plt.ylim(ylim[0],ylim[1])

#plt.savefig('figures/nac_exp_correlation.eps')

#### Correlation of G_NWB with experimental G_cat

In [ ]:
NWB=G_all_mean[:,-2]
NAC=G_all_mean[:,-1]
nNAC=Gcat-G_all_mean[:,-1]

y_fit, popt, r2 = linear_fit(NWB, Gcat)
print(r2)

### Energy Band Analysis

In [ ]:
#Non nac contributions
Gelec=Gcat-comp
#Max and min barriers from kcat values in literature - from Table in Reaction kinetics paper
Gcat_max=kBT*np.log(kBT_over_h/0.09)+kBT
Gcat_min=kBT*np.log(kBT_over_h/114)-kBT

xlim=[-1,13]
ylim=[0,19]
junction_list=['MA-CA','CA-SP1','SP1-NC','NC-SP2','SP2-p6','TFR-PR','PR-RT','RT-RH','RH-IN']
index = 1.2*np.arange(len(junction_list)) + 0.3
bar_width = 0.8
# Initialize the vertical-offset for the stacked bar chart.
y_offset = np.zeros(len(junction_list))

xline_x=[x for x in range(xlim[0],xlim[1]+1)]
xline_y=np.ones((1,xlim[1]-xlim[0]+1))[0]
band_vals = [ np.mean(np.sort(Gelec)[0:2]), np.mean(np.sort(Gelec)[2:8]), np.sort(Gelec)[8] ]

band=[]
for i in range(len(band_vals)):
    band.append(band_vals[i]*xline_y)
error_band=kBT*xline_y

#fig, ax = plt.subplots()
plt.figure(figsize=(15,9))
plt.rc('text', usetex=True)

#kcat physiological region 
plt.fill_between(xline_x, Gcat_min*xline_y, Gcat_max*xline_y, color='#fee090',alpha=0.3,zorder=0)

#Discreted bands reached by non-nac contributions
for i in range(len(band)):
    plt.plot(xline_x, band[i], ls='-',color='k',lw='4',zorder=5)
    if i==0:
        plt.fill_between(xline_x, band[i]-error_band, band[i]+error_band, color='#d1e5f0',alpha=0.6,zorder=5)
    elif i==1:
        plt.fill_between(xline_x, band[i]-error_band, band[i]+error_band, color='#67a9cf',alpha=0.6,zorder=5)
    else:
        plt.fill_between(xline_x, band[i]-error_band, band[i]+error_band, color='#2166ac',alpha=0.6,zorder=5)
        
#Order of bars
order=np.argsort(Gelec)


#Stacked bar chart of nac and non-nac contributions    
plt.bar(index[0:2], Gelec[order[0:2]], bar_width, bottom=y_offset[0:2], color='#d1e5f0',zorder=10)
plt.bar(index[2:8], Gelec[order[2:8]], bar_width, bottom=y_offset[2:8], color='#67a9cf',zorder=10)
plt.bar(index[8], Gelec[order[8]], bar_width, bottom=y_offset[8], color='#2166ac',zorder=10)

y_offset = y_offset + Gelec[order]
plt.bar(index, comp[order], bar_width, bottom=y_offset, color='#b2182b',zorder=10)


#plt.xlabel(r'$d_{n} (\AA)$',fontsize=30)
plt.ylabel(r'$\Delta G^{\ddagger}_{cat} (kcat/mol)$',fontsize=30)
plt.xticks(index, [junction_list[x] for x in order], fontsize=20)  
plt.yticks([y for y in range(0,22,2)], fontsize=30)
plt.xlim(xlim[0],xlim[1])
plt.ylim(ylim[0],ylim[1])


plt.text(11, 16, "{:.1f}".format(band_vals[2]), fontsize=25)
plt.text(11, 13.5, "{:.1f}".format(band_vals[1]), fontsize=25)
plt.text(11, 11.5, "{:.1f}".format(band_vals[0]), fontsize=25)

plt.text(11, 7, '$\Delta G^{\ddagger}_{NAC}$', fontsize=25,color='#b2182b')
plt.text(11, 3, '$\Delta G^{\ddagger}_{nonNAC}$', fontsize=25,color='b')

#d1e5f0
#67a9cf
#2166ac

plt.show()

#plt.savefig('figures/energy_bands.eps')

print(Gcat)
print(Gcat_min, Gcat_max)
print(comp)
print(Gelec)
print(band_vals)

#print(np.sort(Gelec))
#print(np.mean(np.sort(Gelec)[0:2]),np.std(np.sort(Gelec)[0:2]))
#print(np.mean(nsp.sort(Gelec)[2:8]),np.std(np.sort(Gelec)[2:8]))
#print(np.sort(Gelec)[8])



### Combined Plot of Correlation and Energy bands

In [ ]:

junction_list=['MA-CA','CA-SP1','SP1-NC','NC-SP2','SP2-p6','TFR-PR','PR-RT','RT-RH','RH-IN']

#fig, axes = plt.subplots(1, 2, sharex=False,sharey=False,figsize=(17,5))

fig = plt.figure(figsize=(17, 5)) 
fig.subplots_adjust(hspace=0.05, wspace=0.3)
gs = gridspec.GridSpec(1, 2, width_ratios=[1, 1.5])
gs.update(left=0.1,right=0.9,top=0.98,bottom=0.25,wspace=0.3,hspace=0.05)
ax0 = plt.subplot(gs[0])
ax1 = plt.subplot(gs[1])


#Figure A
#Using data in Maschera et al
#Then using ratio of NC-SP2/CA-SP1 in Feher and Toszer parameters to calculate NC-SP2 equivalent in Maschera conditions
kcat=[[23, 2], [2.6, 0.1], [41, 6], [5*2.6/3, 0.1], [0.25, 0.1], [43, 4], [20, 3], [1.8, 0.1], [5.1, 0.1]] #s^-1
kcat=np.array(kcat)         
Gcat=kBT*np.log(kBT_over_h/kcat[:,0])
comp=G_all_mean[:,-1]
comp_err=G_all_std[:,-1]

print(Gcat)
print(comp)
print(comp_err)

def func(x, a, b):

    return a *x + b

popt, pcov = curve_fit(func, Gcat, comp)
y_fit=func(Gcat, *popt)

# residual sum of squares
ss_res = np.sum((comp - y_fit) ** 2)
# regression sum of squares
ss_reg = np.sum((y_fit - np.mean(comp)) ** 2) 
# total sum of squares
ss_tot = np.sum((comp - np.mean(comp)) ** 2)
# r-squared
#r2 = 1 - (ss_res / ss_tot)
r2 = (ss_reg / ss_tot)

#print(popt,r2)



xlim=[14,19]
ylim=[0,6]
#axes[0].rc('text', usetex=True)
ax0.text(14.2, 5.4, '(A)', fontsize=30,color='k')
ax0.errorbar(Gcat,comp,comp_err,ls='',color='k',marker='o',ms=14, mec='k',mfc='#b2182b')
ax0.plot(Gcat, y_fit, '-k')
ax0.text(17, 1, '$R^{2}$ =' +  "{:.2f}".format(r2), fontsize=30,color='k')
delx=[-0.2,-0.2,-0.6,-0.6,-0.2,-0.2,-0.1,0.2,-0.2]
dely=[0.3,0.7,0.2,0.3,-0.6,-0.6,0.55,0.3,-0.5]
rot=[0,30,0,0,0,0,0,30,0]
for i in range(len(Gcat)):
    ax0.text(Gcat[i]+delx[i], comp[i]+dely[i], str(junction_list[i]),fontsize=15,color='k',rotation=rot[i])

ax0.set_xlabel(r'$\Delta G^{\ddagger c}$ (kcal/mol)',fontsize=30)
ax0.set_ylabel(r'$\Delta G^{\ddagger c}_{N}$ (kcal/mol)',fontsize=30)
ax0.set_xticks([x/10 for x in range(140,190,10)])
ax0.tick_params(axis='x', labelsize=30)
ax0.set_yticks([y/10 for y in range(0,60,10)])
ax0.tick_params(axis='y', labelsize=30)
ax0.set_xlim(xlim[0],xlim[1])
ax0.set_ylim(ylim[0],ylim[1])

#Figure B

#Non nac contributions
Gelec=Gcat-comp
#Max and min barriers from kcat values in literature - from Table in Reaction kinetics paper
Gcat_max=kBT*np.log(kBT_over_h/0.09)+kBT
Gcat_min=kBT*np.log(kBT_over_h/114)-kBT

xlim=[-1,13]
ylim=[0,19]
index = 1.2*np.arange(len(junction_list)) + 0.3
bar_width = 0.8
# Initialize the vertical-offset for the stacked bar chart.
y_offset = np.zeros(len(junction_list))

xline_x=[x for x in range(xlim[0],xlim[1]+1)]
xline_y=np.ones((1,xlim[1]-xlim[0]+1))[0]
band_vals = [ np.mean(np.sort(Gelec)[0:2]), np.mean(np.sort(Gelec)[2:8]), np.sort(Gelec)[8] ]

band=[]
for i in range(len(band_vals)):
    band.append(band_vals[i]*xline_y)
error_band=kBT*xline_y

#kcat physiological region 
ax1.fill_between(xline_x, Gcat_min*xline_y, Gcat_max*xline_y, color='#fee090',alpha=0.3,zorder=0)

#Discreted bands reached by non-nac contributions
for i in range(len(band)):
    ax1.plot(xline_x, band[i], ls='-',color='k',lw='4',zorder=5)
    if i==0:
        ax1.fill_between(xline_x, band[i]-error_band, band[i]+error_band, color='#d1e5f0',alpha=0.6,zorder=5)
    elif i==1:
        ax1.fill_between(xline_x, band[i]-error_band, band[i]+error_band, color='#67a9cf',alpha=0.6,zorder=5)
    else:
        ax1.fill_between(xline_x, band[i]-error_band, band[i]+error_band, color='#2166ac',alpha=0.6,zorder=5)
        
#Order of bars
order=np.argsort(Gelec)

#Stacked bar chart of nac and non-nac contributions    
ax1.bar(index[0:2], Gelec[order[0:2]], bar_width, bottom=y_offset[0:2], color='#d1e5f0',zorder=10)
ax1.bar(index[2:8], Gelec[order[2:8]], bar_width, bottom=y_offset[2:8], color='#67a9cf',zorder=10)
ax1.bar(index[8], Gelec[order[8]], bar_width, bottom=y_offset[8], color='#2166ac',zorder=10)

y_offset = y_offset + Gelec[order]
ax1.bar(index, comp[order], bar_width, bottom=y_offset, color='#b2182b',zorder=10)


#plt.xlabel(r'$d_{n} (\AA)$',fontsize=30)
ax1.set_ylabel(r'$\Delta G^{\ddagger c}$ (kcal/mol)',fontsize=30)
ax1.set_xticks(index)
ax1.set_xticklabels([junction_list[x] for x in order], rotation=90)
ax1.set_yticks([y for y in range(0,22,2)])
ax1.tick_params(axis='x', labelsize=20)
ax1.tick_params(axis='y', labelsize=30)

ax1.set_xlim(xlim[0],xlim[1])
ax1.set_ylim(ylim[0],ylim[1])


ax1.text(11, 16, "{:.1f}".format(band_vals[2]), fontsize=20)
ax1.text(11, 13.5, "{:.1f}".format(band_vals[1]), fontsize=20)
ax1.text(11, 11.5, "{:.1f}".format(band_vals[0]), fontsize=20)

ax1.text(10.5, 7, '$\Delta G^{\ddagger c}_{N}$', fontsize=20,color='#b2182b')
ax1.text(10.5, 3, '$\Delta G^{\ddagger c}_{nN}$', fontsize=20,color='b')

ax1.text(-0.3, 17, '(B)', fontsize=30, color='k')


#fig.savefig('figures/correl_and_energy_bands.eps')

#fig.savefig('figures/correl_and_energy_bands_new.eps')

#fig.savefig('figures/correl_and_energy_bands.png')
 


#fig.show()

### Hydrogen bond analysis

In [ ]:
#Burgi-Dunitz criteria
BDdist=3.2;
BDanglow=100;
BDanghigh=110;

#Hydrogen bond criteria
hb_dist=3.5
hb_ang=150


foffset=0.00000001

for i in range(np.shape(alldata)[2]):
    data = alldata[:,:,i]

    #Truth array of nac and nwb
    nwb,nac=confs(data,BDdist,BDanglow,BDanghigh,nwb_cut,sys)
    not_nwb=np.logical_not(nwb)
    not_nac=np.logical_not(nac)
    nac_nwb=np.logical_and(nwb,nac)
    not_nac_nwb=np.logical_and(nwb,not_nac)

#print(np.sum(nac), np.sum(not_nac))
#print(np.sum(nwb),np.sum(not_nwb))

    #Truth arrays of all hydrogen bonds
    hb1,hb2,hb3,hb4,hb5,hb6,stat=hbond_analysis(data,hb_dist,hb_ang)
    hb=[hb1, hb2, hb3, hb4]
    #List of truth arrays of all chosen hbond combinations - termed hbond states
    hbs,lst=hb_states(hb)

    #Filtereing hbond states by nac,nwb combinations
    hbs_nac_nwb=[np.logical_and(x,nac_nwb) for x in hbs]
    hbs_not_nac_nwb=[np.logical_and(x,not_nac_nwb) for x in hbs]
    hbs_not_nwb=[np.logical_and(x,not_nwb) for x in hbs]
    
    hbs_nac=[np.logical_and(x,nac) for x in hbs]

    
    
#print(lst)
#print([np.sum(x) for x in hbs])
    rho=[]
    for x in range(len(lst)):
        f=[np.sum(hbs_nac_nwb[x])/np.sum(nac_nwb), np.sum(hbs_not_nac_nwb[x])/np.sum(not_nac_nwb), np.sum(hbs_not_nwb[x])/np.sum(not_nwb)]
        f=[x+foffset for x in f]        
        #f=[np.sum(hbs_nac[x])/np.sum(nac), np.sum(hbs_not_nac_nwb[x])/np.sum(not_nac_nwb), np.sum(hbs_not_nwb[x])/np.sum(not_nwb)]
        rho.append(f)    
        #print(lst[x],f)
    
    rho=np.array(rho) 

    #This is the order of states 1-12 in the NAC paper
    hb_order=np.array([0, 2, 1, 3, 8, 10, 9, 11, 4, 6, 5, 7])
    #print(rho[hb_order,:])
    rho_mat=rho[hb_order,:]
    rho_mat=np.flipud(np.transpose(rho_mat))
    #print(np.sum(rho_mat,axis=0))
    G=-kBT*np.log(rho_mat)
    #print(G)
    #G=np.flipud(np.transpose(G))

    if i==0:
        Gall=G
        rho_all=rho_mat
    else:
        Gall=np.dstack((Gall,G))
        rho_all=np.dstack((rho_all,rho_mat))

#Test of the cbar axis - which was successful below
#Gall[:,:,8]=3*np.random.rand(3,12)        

#### Plot Hydrogen-bond analysis matrix

In [ ]:
#fig,ax=plt.subplots()

junction_list=['MA-CA','CA-SP1','SP1-NC','NC-SP2','SP2-p6','TFR-PR','PR-RT','RT-RH','RH-IN']
index=[x for x in range(12)]
xlim=[index[0]-0.5,index[-1]+0.5]
ylim=[-0.5,2.5]

fig, axes = plt.subplots(3, 3, sharex=False,sharey=True,figsize=(15,6))
fig.subplots_adjust(hspace=0.05, wspace=0.05)



for i in range(3):
    for j in range(3):
        sys=3*i + j
        
        hbmap=axes[i,j].imshow(Gall[:,:,sys],interpolation=None, cmap='inferno', vmin=0, vmax=6)


        axes[i, j].set_xticks([x for x in index])
        axes[i, j].set_xticklabels([x+1 for x in index])
        axes[i, j].set_yticks([0,1,2])
        axes[i, j].set_yticklabels(['Unbound','GS','NAC'])
        
        
        
        axes[i, j].tick_params(axis='x', labelsize=15)
        axes[i, j].tick_params(axis='y', labelsize=15)
        
        axes[i, j].text(4.0, 3.0, junction_list[sys],fontsize=20)
        
        axes[i, j].set_xlim(xlim[0],xlim[1])
        axes[i, j].set_ylim(ylim[0],ylim[1])
        
        
axes[2,1].set_xlabel(r'H-bond states',fontsize=30)
axes[1,0].set_ylabel(r'Conformation',fontsize=30)


cbar=fig.colorbar(hbmap, ax=axes[:, :], shrink=0.8)
cbar.set_label('G (kcal/mol)', rotation=90,fontsize=30, labelpad=20)
cbar.ax.tick_params(labelsize=20)
cbar.ax.set_yticklabels([x for x in range(7)])
#cbar.set_clim(0.0, 6.0)

plt.show()

#fig.savefig('figures/hbond_network.eps')

## Final  Results Table

In [ ]:
np.set_printoptions(precision=1)

dG_NWB=G_all_mean[:,-2]
dG_NWB_er=G_all_std[:,-2]

dGu=30
dGu_N=G_all_mean_apo[:,-1]
dGu_N_er=G_all_std_apo[:,-1]
dGu_nN=dGu-dGu_N

dGu_ave=np.array([[np.mean(dGu_N),np.std(dGu_N)], [np.mean(dGu_nN),np.std(dGu_nN)]])

dGc=Gcat
dGc_N=G_all_mean[:,-1]
dGc_N_er=G_all_std[:,-1]
dGc_nN=dGc-dGc_N

ddG=dGc-dGu
ddG_N=dGc_N-dGu_N
ddG_N_er=dGu_N_er+dGc_N_er
ddG_nN=dGc_nN-dGu_nN

print(dG_NWB)
print(dG_NWB_er)

print(dGu_N)
print(dGu_N_er)
print(dGu_nN)

print(dGc)
print(dGc_N)
print(dGc_N_er)
print(dGc_nN)

print(ddG)
print(ddG_N)
print(ddG_N_er)
print(ddG_nN)


print(dGu_ave)
